In [ ]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting up displays
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from dash import Dash,html,dcc,Input,Output
app = Dash(__name__)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
from dash import Dash,html,dcc

In [ ]:
# ##center allign all the figure outputs.
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# from IPython.display import display, HTML
# from plotly.graph_objs import *
# import numpy as np
# init_notebook_mode(connected=True)

# display(HTML("""
# <style>
# .output {
#     display: flex;
#     align-items: center;
#     text-align: center;
# }
# </style>
# """))

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.queries

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-10-01'
edate = '2022-10-31'
print(f"The analysis is carried our for date range {sdate} to {edate}")

# Query Analysis

## Most expensive queries

In [ ]:
df=qlib.n_expensive_queries(sdate,edate,5)
df.head()

In [ ]:
##TODO: Put labels on the axis
## Most expensive queries

trace1  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['credits'],
        name="Credits",
        marker_color='crimson'
    )

# trace2 = go.Bar(
#         x = df['query_id'],
#         y = df['warehouse_size'],
#         name="warehouse size",
#         yaxis='y2',
#         marker_color ='green',
#         marker_line_width=1.5,
#         marker_line_color='rgb(8,48,107)',
#         opacity=0.5
#     )

# data = [trace1, trace2]
data = [trace1]

layout = go.Layout(
    title_text='Most expensive queries',
    yaxis=dict(
        # range = [0, 100],
        side = 'left'
    ),
    # yaxis2=dict(
    #     overlaying='y',
    #     anchor='y3',
    # )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

### Query text

#### Query 1

In [ ]:
print(df.iloc[0]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[2]["query_text"])

## Queries that spill to storage

In [ ]:
df=qlib.n_queries_spill_to_storage(sdate,edate,5)
df.head()

In [ ]:
##TODO: Put labels on the axis
## Most expensive queries

trace1  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['bytes_spilled_to_remote_storage'],
        name="Bytes Spilled Remote",
        marker_color='crimson'
    )

trace2  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['bytes_spilled_to_local_storage'],
        name="Bytes Spilled Local",
        marker_color='purple'
    )


data = [trace1, trace2]


layout = go.Layout(
    title_text='Queries that spilled the most to storage',
    yaxis=dict(
        # range = [0, 100],
        side = 'left'
    ),
    yaxis2=dict(
        overlaying='y',
        anchor='y3',
    )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

### Query text

#### Query 1

In [ ]:
print(df.iloc[0]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[2]["query_text"])

## Queries that scanned the most data

In [ ]:
df=qlib.n_queries_scanned_most_data(sdate,edate,5)
df.head() 

In [ ]:
##TODO: Put labels on the axis
## Queries that scanned most data

trace1  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['partitions_scanned'],
        name="Partitions Scanned",
        marker_color='crimson'
    )

data = [trace1]

layout = go.Layout(
    title_text='Queries that scanned the most partitions',
    yaxis=dict(
        side = 'left'
    ),
)
fig = go.Figure(data=data, layout=layout)
fig.show()

### Query text

#### Query 1

In [ ]:
print(df.iloc[0]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[2]["query_text"])

## Most cached queries

In [ ]:
df=qlib.n_most_cached_queries(sdate,edate,5)
df.head() 

In [ ]:
##TODO: Put labels on the axis
## Queries that scanned the most from cache

trace1  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['percent_scanned_from_cache'],
        name="Percent Scanned From Cache",
        marker_color='crimson'
    )

data = [trace1]

layout = go.Layout(
    title_text='Queries that scanned the most percent from cache',
    yaxis=dict(
        side = 'left'
    ),
)
fig = go.Figure(data=data, layout=layout)
fig.show()

#### Query 1

In [ ]:
print(df.iloc[2]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[2]["query_text"])

## Most executed 'select' queries -- update this for select statements

In [ ]:
df=qlib.n_most_executed_queries(sdate,edate,10)
df.head() 

In [ ]:
##TODO: Put labels on the axis, fix ticks on the axis to be integers
## Queries that scanned the most from cache

trace1  = go.Scatter(
        mode='lines+markers',
        x = df.index,
        y = df['number_of_times_executed'],
        name="Number of times executed",
        marker_color='crimson'
    )

data = [trace1]

layout = go.Layout(
    title_text='Queries that were executed the most number of times',
    yaxis=dict(
        side = 'left'
    ),
)
fig = go.Figure(data=data, layout=layout)
fig.show()

#### Query 1

In [ ]:

print(df.iloc[0]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[3]["query_text"])